In [1]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2 as cv
import matplotlib.pyplot as plt

from scipy.spatial import distance as dist

In [2]:
def eye_detect(eye):
    x1 = dist.euclidean(eye[1], eye[5])
    x2 = dist.euclidean(eye[2], eye[4])
    x3 = dist.euclidean(eye[0], eye[3])

    ear = (x1+x2) / (2*x3)
    return ear

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
thresh_blink = 0.34
consec_frames = 3

count = 0
tot = 0

In [5]:
(l_start, l_end) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(r_start, r_end) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [6]:
capture = cv.VideoCapture(0)
cod = cv.VideoWriter_fourcc(*'XVID')

if (capture.isOpened() == False): 
    print("Error in opening webcam")

w  = int(capture.get(3))
h = int(capture.get(4))
fps = int(capture.get(5))

video = cv.VideoWriter(r"C:\Users\MSI\Desktop\Face\blink_detection.mp4", cod, fps/4, (w,h))

In [7]:
while True:
    if capture.get(cv.CAP_PROP_POS_FRAMES) == capture.get(cv.CAP_PROP_FRAME_COUNT):
        capture.set(cv.CAP_PROP_POS_FRAMES, 0)
    else:
        ret, im = capture.read()
        im = imutils.resize(im, width=w)
        im_gray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)

        rects = detector(im_gray, 1)
        for rect in rects:
            ident = predictor(im_gray, rect)
            ident = face_utils.shape_to_np(ident)

            leftEye = ident[l_start:l_end]
            rightEye = ident[r_start:r_end]
            left_EAR = eye_detect(leftEye)
            right_EAR = eye_detect(rightEye)

            avg_ear = (left_EAR + right_EAR) / 2.0
            left_Hull = cv.convexHull(leftEye)
            right_Hull = cv.convexHull(rightEye)
            cv.drawContours(im, [left_Hull], -1, (0, 255, 0), 1)
            cv.drawContours(im, [right_Hull], -1, (0, 255, 0), 1)

            if avg_ear < thresh_blink:
                count += 1

            else:
                if count >= consec_frames:
                    tot += 1
                    cv.putText(im, "Blink Detected", (200, 30), cv.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 3)

                count = 0

            cv.putText(im, "Blinks: {}".format(tot), (10, 30), cv.FONT_HERSHEY_DUPLEX, 0.75, (0, 0, 255), 2)
            cv.putText(im, "EAR: {:.2f}".format(avg_ear), (500, 30), cv.FONT_HERSHEY_DUPLEX, 0.75, (0, 0, 255), 2)
        cv.imshow("Frame", im)

        if cv.waitKey(1) & 0xFF == ord('x'):
            break

    video.write(im)


capture.release()
video.release()
cv.destroyAllWindows()
    